# 1.模型部署简介

模型部署： 算法模型 -->  应用程序

希望通过本教程，带领大家学会如何把自己的 PyTorch 模型部署到 ONNX Runtime/TensorRT 上。

* 中间表示 ONNX 的定义标准
* PyTorch 模型转换到 ONNX 模型的方法
* 推理引擎 ONNX Runtime、TensorRT 的使用方法
* 部署流水线 PyTorch - ONNX - ONNX Runtime/TensorRT 的示例及常见部署问题的解决方法
* MMDeploy C/C++ 推理 SDK


软件工程领域的“部署”：把开发完毕的软件投入使用的过程，包括环境配置、软件安装等。对于深度学习模型而言，模型部署指让训练好的模型在特定的环境中运行的过程。这过程中需要解决：环境适配、运行性能等问题。

部署流水线：

1. 训练：使用深度学习框架训练模型的参数，训练框架包括了：Pytorch、TensorFlow、Caffe、MXNet等
2. 中间表示（优化）：将不同框架下训练的模型结构定义和权重参数，转换为中间表达，常见的中间表达有：ONNX、TorchScript、Caffe等
3. 推理引擎（运行）：将中间表示转化为特定的优化的推理引擎的专用格式。常见的推理引擎有：TensorRT、ONNXRuntime、PPL、NCNN、OpenVINO等。

# 2.部署第一个模型

Pytorch Module -> ONNX -> ONNX Runtime

## 2.1 定义一个简单的图像超分的模型

该超分模型由一个上采样层和3层卷积层构成。

In [1]:
import os
import cv2
import numpy as np

import requests
import torch
import torch.onnx
from torch import nn


class SuperResolutionNet(nn.Module):
    def __init__(self, upscale_factor):
        super().__init__()
        self.upsampler = nn.Upsample(
            scale_factor=upscale_factor, mode="bicubic", align_corners=False
        )
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, padding=0)  # 压缩channel
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)

        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.upsampler(x)
        out = self.relu(self.conv1(out))
        out = self.relu(self.conv2(out))
        out = self.conv3(out)
        return out

In [2]:
model = SuperResolutionNet(2.0)
x = torch.randn((1, 3, 224, 224))
for modules in model.named_modules():
    if modules[0] == "" or modules[0] == "relu":
        continue
    shape = x.shape
    x = model.get_submodule(modules[0])(x)
    print(f"{modules[0]}: {list(shape)} => {list(x.shape)}")

upsampler: [1, 3, 224, 224] => [1, 3, 448, 448]
conv1: [1, 3, 448, 448] => [1, 64, 448, 448]
conv2: [1, 64, 448, 448] => [1, 32, 448, 448]
conv3: [1, 32, 448, 448] => [1, 3, 448, 448]


## 2.2 加载训练好的模型权重

In [3]:
urls = [
    "https://download.openmmlab.com/mmediting/restorers/srcnn/srcnn_x4k915_1x16_1000k_div2k_20200608-4186f232.pth",
    "https://raw.githubusercontent.com/open-mmlab/mmediting/master/tests/data/face/000001.png",
]

names = ["srcnn.pth", "face.png"]

for url, name in zip(urls, names):
    if not os.path.exists(name):
        open(name, "wb").write(requests.get(url).content)

# 加载离线权重，并对key进行适配
state_dicts = torch.load("srcnn.pth")["state_dict"]
print(state_dicts.keys())
for old_key in list(state_dicts.keys()):
    new_key = old_key.replace("generator.", "", 1)
    state_dicts[new_key] = state_dicts.pop(old_key)
print(state_dicts.keys())
srnn_model = SuperResolutionNet(upscale_factor=3)
srnn_model.load_state_dict(state_dict=state_dicts)

odict_keys(['generator.conv1.weight', 'generator.conv1.bias', 'generator.conv2.weight', 'generator.conv2.bias', 'generator.conv3.weight', 'generator.conv3.bias'])
odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'conv3.weight', 'conv3.bias'])


<All keys matched successfully>

## 2.3 使用Pytorch进行推理

In [4]:
input_image = cv2.imread("./face.png").astype(np.float32)
input_image = np.transpose(input_image, (2, 0, 1))  # H,W,C -> C,H,W
input_image = np.expand_dims(input_image, 0)  # C,H,W -> 1,C,H,W

torch_output = srnn_model(torch.from_numpy(input_image)).detach().numpy()
torch_output = np.squeeze(torch_output, 0)
torch_output = np.clip(torch_output, 0, 255)
torch_output = np.transpose(torch_output, (1, 2, 0)).astype(np.uint8)

cv2.imwrite("./outputs/face_torch_1.png", torch_output)

True

## 2.4 导出为ONNX格式的模型

In [5]:
import onnx

x = torch.randn(1, 3, 256, 256)
with torch.no_grad():
    torch.onnx.export(
        model=srnn_model,
        args=x,
        f="./outputs/srcnn1.onnx",
        opset_version=11,
        input_names=["input"],
        output_names=["output"],
    )

# 使用onnx来加载序列化的模型，加载出来的就是一个ModelProto对象
onnx_model = onnx.load("./outputs/srcnn1.onnx")
# 检查ModelProto是否正常
onnx.checker.check_model(onnx_model, full_check=True)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



## 2.5 使用Netron来查看Onnx文件

地址：[netron.app](https://netron.app/)

<div align="center">
  <img src="./assets/srcnn.onnx.svg" height="600"/> </div>

## 2.6 使用OnnxRuntime进行推理

In [6]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("./outputs/srcnn1.onnx")
ort_inputs = {"input": input_image}

# 输入输出都是numpy.ndarry
ort_output = ort_session.run(output_names=["output"], input_feed=ort_inputs)[0]

ort_output = np.squeeze(ort_output, 0)
ort_output = np.clip(ort_output, 0, 255)
ort_output = np.transpose(ort_output, (1, 2, 0)).astype(np.uint8)

successed = cv2.imwrite("./outputs/face_ort_1.png", ort_output)

# 3. 解决模型部署中的难题

上面部署的模型，十分顺利，但是它有一个缺陷，图片的放大倍数被写死在模型结构中了，我们在推理时，无法动态的调整。

另外上面模型部署的简单也主要是因为SCRNN结构比较简单，只包括了：`Upsample`、`Conv2d`、`Relu`这3个标准的算子。这些算法在各个中间表示和推理引擎上已经得到完美的支持。如果模型的操作稍微复杂一点，我们可能就要为兼容模型而付出大量的功夫了。在实际部署时，我们可能会遇到几下几类的困难：

* 模型的动态化。出于性能的考虑，各推理框架都默认模型的输入形状、输出形状、结构是静态的。而为了让模型的泛化性更强，部署时需要在尽可能不影响原有逻辑的前提下，让模型的输入输出或是结构动态化。
* 新算子的实现。深度学习技术日新月异，提出新算子的速度往往快于 ONNX 维护者支持的速度。为了部署最新的模型，部署工程师往往需要自己在 ONNX 和推理引擎中支持新算子。
* 中间表示与推理引擎的兼容问题。由于各推理引擎的实现不同，对 ONNX 难以形成统一的支持。为了确保模型在不同的推理引擎中有同样的运行效果，部署工程师往往得为某个推理引擎定制模型代码，这为模型部署引入了许多工作量。

## 3.1 SRCNN支持动态设置放大倍数

下面我们对前面的`SRCNN`网络结构进行修改，将上采样写在`forward`阶段，调用`interpolate`函数来实现。

In [7]:
import torch.nn.functional as F


class SuperResolutionNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, padding=0)  # 压缩channel
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)

        self.relu = nn.ReLU()

    def forward(self, x, upscale_factor):
        out = F.interpolate(
            x, scale_factor=upscale_factor, mode="bicubic", align_corners=False
        )
        out = self.relu(self.conv1(out))
        out = self.relu(self.conv2(out))
        out = self.conv3(out)
        return out


torch_model = SuperResolutionNet()
torch_model.load_state_dict(state_dicts)
torch_model.eval()

SuperResolutionNet(
  (conv1): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
  (conv2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
  (conv3): Conv2d(32, 3, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu): ReLU()
)

In [8]:
upscale_factor = 3.0
torch_output = (
    torch_model(torch.from_numpy(input_image), upscale_factor).detach().numpy()
)
torch_output = np.squeeze(torch_output, 0)
torch_output = np.clip(torch_output, 0, 255)
torch_output = np.transpose(torch_output, (1, 2, 0)).astype(np.uint8)
cv2.imwrite("./outputs/face_torch_2.png", torch_output)

True

由于pytorch本身具有动态性，所以上面的过程没有任何问题，我们在forward时，指定了放大倍数。现在我们来尝试将上面的模型转换为ONNX模型。

In [9]:
with torch.no_grad():
    torch.onnx.export(
        model=torch_model,
        args=(x, 3),
        f="./outputs/srcnn2.onnx",
        opset_version=11,
        input_names=["input", "factor"],
        output_names=["output"],
    )

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



TypeError: upsample_bicubic2d() received an invalid combination of arguments - got (Tensor, NoneType, bool, list), but expected one of:
 * (Tensor input, tuple of ints output_size, bool align_corners, tuple of floats scale_factors)
      didn't match because some of the arguments have invalid types: (Tensor, !NoneType!, bool, !list of [Tensor, Tensor]!)
 * (Tensor input, tuple of ints output_size, bool align_corners, float scales_h, float scales_w, *, Tensor out)


上面报错的原因是，在导出到ONXX模型时，模型的输入参数的类型必须全部是torch.Tensor。而我们传入的第二个参数“3”是一个整形变量。这不符合Pytorch转ONNX的规定。现在我们可以尝试修改一下模型：

In [10]:
class SuperResolutionNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, padding=0)  # 压缩channel
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)

        self.relu = nn.ReLU()

    def forward(self, x, upscale_factor):
        out = F.interpolate(
            x, scale_factor=upscale_factor.item(), mode="bicubic", align_corners=False
        )
        out = self.relu(self.conv1(out))
        out = self.relu(self.conv2(out))
        out = self.conv3(out)
        return out


torch_model = SuperResolutionNet()
torch_model.load_state_dict(state_dicts)
torch_model.eval()

with torch.no_grad():
    torch.onnx.export(
        model=torch_model,
        args=(x, torch.tensor(3)),
        f="./outputs/srcnn2.onnx",
        opset_version=11,
        input_names=["input", "factor"],
        output_names=["output"],
    )

/tmp/ipykernel_3137538/646510743.py:12: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x, scale_factor=upscale_factor.item(), mode="bicubic", align_corners=False


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



转换成功了，但报了一个Warning，意思是虽然转成功了，但在转换过程中，发现了把一个Tensor转换为了一个Python number，这会导致导出的ONNX模型，把这个值视为一个常量。也就是说导出的模型，我们还是无法动态的设置upscale_factor。

如果我们在netron.app中加载刚才导出来的ONNX文件，会发现，还是只有一个输入。

## 3.2 自定义ONNX算子解决问题

仔细观察 Netron 上可视化出的 ONNX 模型，可以发现在 PyTorch 中无论是使用最早的 nn.Upsample，还是后来的 interpolate，PyTorch 里的插值操作最后都会转换成 ONNX 定义的 Resize 操作。也就是说，所谓 PyTorch 转 ONNX，实际上就是把每个 PyTorch 的操作映射成了 ONNX 定义的算子。

其中`Resize`算子的输入是：`X`、`roi`、`scales`。`scales`是一个长度为4的一维张量，其内容为`[1,1,3,3]`代表的是各个维度的缩放系数。下面我们来尝试自定义一个pytorch的算子，希望把它映射到ONXX的Resize算子上。

In [11]:
class NewInterpolate(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, scales):
        scales = scales.tolist()[-2:]
        return F.interpolate(
            input, scale_factor=scales, mode="bicubic", align_corners=False
        )

    @staticmethod
    def symbolic(g, input, scales):
        return g.op(
            "Resize",
            input,  # 第一个参数 X
            g.op(
                "Constant", value_t=torch.tensor([], dtype=torch.float32)
            ),  # 第二个参数 roi
            scales,  # 第三个参数  scales
            coordinate_transformation_mode_s="pytorch_half_pixel",
            cubic_coeff_a_f=-0.75,
            mode_s="cubic",
            nearest_mode_s="floor",
        )

映射到 ONNX 的方法由一个算子的 `symbolic` 方法决定。`symbolic` 方法第一个参数必须是`g`，之后的参数是算子的自定义输入，和 `forward` 函数一样。ONNX 算子的具体定义由 `g.op` 实现。`g.op` 的每个参数都可以映射到 ONNX 中的算子属性：

<div align="center">
  <img src="./assets/symbolic_onnx_op.png" height="400"/> </div>

In [12]:
class StrangeSuperResolutionNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, padding=0)  # 压缩channel
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)

        self.relu = nn.ReLU()

    def forward(self, x, upscale_factor):
        out = NewInterpolate.apply(x, upscale_factor)
        out = self.relu(self.conv1(out))
        out = self.relu(self.conv2(out))
        out = self.conv3(out)
        return out


torch_model = StrangeSuperResolutionNet()
torch_model.load_state_dict(state_dicts)
torch_model.eval()


upscale_factor = torch.tensor([1, 1, 3, 3], dtype=torch.float32)
torch_output = (
    torch_model(torch.from_numpy(input_image), upscale_factor).detach().numpy()
)
torch_output = np.squeeze(torch_output, 0)
torch_output = np.clip(torch_output, 0, 255)
torch_output = np.transpose(torch_output, (1, 2, 0)).astype(np.uint8)
cv2.imwrite("./outputs/face_torch_3.png", torch_output)

with torch.no_grad():
    torch.onnx.export(
        model=torch_model,
        args=(x, upscale_factor),
        f="./outputs/srcnn3.onnx",
        opset_version=11,
        input_names=["input", "factor"],
        output_names=["output"],
    )

/tmp/ipykernel_3137538/3127156981.py:4: TracerWarning: Converting a tensor to a Python list might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  scales = scales.tolist()[-2:]


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



现在我们已经成功导出来了新的ONXX模型，虽然上面依然有一条Tracing的Warning，但我们通过Netron.APP看到导出的ONNX算子，已经支持两个输入了。

<div align="center">
  <img src="./assets/new_onnx_model.png" height="400"/> </div>

In [ ]:
ort_session = onnxruntime.InferenceSession("./outputs/srcnn3.onnx")

ort_inputs = {"input": input_image, "factor": upscale_factor.numpy()}

# 输入输出都是numpy.ndarry
ort_output = ort_session.run(output_names=["output"], input_feed=ort_inputs)[0]

ort_output = np.squeeze(ort_output, 0)
ort_output = np.clip(ort_output, 0, 255)
ort_output = np.transpose(ort_output, (1, 2, 0)).astype(np.uint8)

successed = cv2.imwrite("./outputs/face_ort_3.png", ort_output)

OK，导出的ONNX模型在ONNXRuntime中也能成功运行了；回顾一下，最开始不能直接导出的主要原因是，原始的`interpolate`算子的第二个参数是一个python number，它在向ONNX进行算子转换时，会被当作一个Çonstant进行转换。所以如果能把第二个参数改为一个`Tensor`就可以了，所以我们手动定义了一个WrapInterpolate函数，并通完了`symbolic`来手动定义它与ONNX算子之间的映射关系。

# 4. Pytorch模型转ONNX模型

ONNX 是目前模型部署中最重要的中间表示之一。学懂了 ONNX 的技术细节，就能规避大量的模型部署问题。

* 第一篇文章里，我们会介绍更多 PyTorch 转 ONNX 的细节，让大家完全掌握把简单的 PyTorch 模型转成 ONNX 模型的方法；
* 在第二篇文章里，我们将介绍如何在 PyTorch 中支持更多的 ONNX 算子，让大家能彻底走通 PyTorch 到 ONNX 这条部署路线；
* 第三篇文章里，我们讲介绍 ONNX 本身的知识，以及修改、调试 ONNX 模型的常用方法，使大家能自行解决大部分和 ONNX 有关的部署问题。

## 4.1 `torch.onnx.export`详解

在把 PyTorch 模型转换成 ONNX 模型时，我们往往只需要轻松地调用一句`torch.onnx.export`就行了。这个函数的接口看上去简单，但它在使用上还有着诸多的“潜规则”。

`torch.onnx.export`把一个Pytorch Module转换为一个ONNX模型，实际是经过了2步：

1. 先把动态图转换为`torch.jit.ScriptModule`。
2. 再导出为ONNX模型。

如果我们传给`torch.onnx.export`接口的是一个普通的`torch.nn.Module`，那么会先通过`torch.jit.trace`方式把模型变为`torch.jit.ScriptModule`。所以我们也可以直接给接口传入一个已经转好的`torch.jit.ScriptModule`。

<div align="center">
  <img src="./assets/torch_onnx_export.drawio.svg" width="800"/> </div>


我们在进行实际的模型部署时，不需要显式的把pytorch模型转换为TorchScript模型，直接调用`export`接口导出即可，但了解上面的过程，有利于我们在导出时遇到报错时，可以分析问题发生在哪一步。

In [5]:
import torch
import torch.nn as nn


class Model(torch.nn.Module):
    def __init__(self, n):
        super().__init__()
        self.n = n
        self.conv = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3)

    def forward(self, x):
        for _ in range(self.n):
            x = self.conv(x)
        return x


model = Model(n=3)
dummy_input = torch.randn(1, 3, 10, 10)
trace_model = torch.jit.trace(model, example_inputs=dummy_input)
torch.onnx.export(trace_model, [dummy_input], "./outputs/trace_model.onnx")

script_model = torch.jit.script(model)
torch.onnx.export(script_model, [dummy_input], "./outputs/script_model.onnx")

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



/home/yangyansheng/miniconda3/envs/pyml/lib/python3.8/site-packages/torch/onnx/utils.py:825: UserWarning: no signature found for <torch.ScriptMethod object at 0x7f6067655a90>, skipping _decide_input_format
  warnings.warn(f"{e}, skipping _decide_input_format")


上面两种方式导出的ONNX模型可视化出现没有区别，但是按原文档，script model导出的模型，最终的 ONNX 模型用 Loop 节点来表示循环。不清楚是不是因为版本的原因导致的。

下面我们来从应用的角度来介绍每个参数在不同模型部署场景中该如何设置。

```python
def export(
    # 要导出的模型，目前nn.Module和ScripModule
    model: Union[torch.nn.Module, torch.jit.ScriptModule, torch.jit.ScriptFunction],
    # 模型的输入，如果只有一个参数，则为Tensor，如果多个输入，则是一个Tuple
    args: Union[Tuple[Any, ...], torch.Tensor],
    # 导出的ONXX的文件路径，或者BytesIO
    f: Union[str, io.BytesIO],
    # 在导出模型结构时，是否同时导出模型参数，一般都设置为True
    # False 当我们只为了在不同框架传递模型时，可能会用上，一般模型时用不上
    export_params: bool = True,
    verbose: bool = False,
    training: _C_onnx.TrainingMode = _C_onnx.TrainingMode.EVAL,
    # 设置输入和输出张量的名称。如果不设置的话，会自动分配一些简单的名字（如数字）。
    # 很多推理框架在推理ONNX时，都会以{"name": Tensor}来作为输入
    input_names: Optional[Sequence[str]] = None,
    # 如 input_names
    output_names: Optional[Sequence[str]] = None,
    operator_export_type: _C_onnx.OperatorExportTypes = _C_onnx.OperatorExportTypes.ONNX,
    opset_version: Optional[int] = None,
    do_constant_folding: bool = True,
    dynamic_axes: Optional[
        Union[Mapping[str, Mapping[int, str]], Mapping[str, Sequence[int]]]
    ] = None,
    keep_initializers_as_inputs: Optional[bool] = None,
    custom_opsets: Optional[Mapping[str, int]] = None,
    export_modules_as_functions: Union[bool, Collection[Type[torch.nn.Module]]] = False,
) -> None:
```

重要的几个参数，已经在上面接口里做了注释。另外两个重要参数，需要重点介绍一下：

**dynamic_axes**

指定输入输出张量的哪些维度是动态的。 为了追求效率，ONNX 默认所有参与运算的张量都是静态的（张量的形状不发生改变）。但在实际应用中，我们又希望模型的输入张量是动态的，尤其是本来就没有形状限制的全卷积模型。因此，我们需要显式地指明输入输出张量的哪几个维度的大小是可变的。

In [10]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3)

    def forward(self, x):
        return self.conv(x)


model = Model()
dummy_input = torch.randn(1, 3, 10, 10)
torch.onnx.export(
    model,
    dummy_input,
    "./outputs/dynamic_input_cnn.onnx",
    input_names=["in"],
    output_names=["out"],
    dynamic_axes={"in": [2, 3], "out": [2, 3]},
)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



/home/yangyansheng/miniconda3/envs/pyml/lib/python3.8/site-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input in
  warnings.warn(
/home/yangyansheng/miniconda3/envs/pyml/lib/python3.8/site-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input out
  warnings.warn(


由于 ONNX 要求每个动态维度都有一个名字，这样写的话会引出一条 UserWarning，警告我们通过列表的方式设置动态维度的话系统会自动为它们分配名字。一种显式添加动态维度名字的方法如下：

In [23]:
torch.onnx.export(
    model,
    dummy_input,
    "./outputs/dynamic_input_cnn.onnx",
    input_names=["in"],
    output_names=["out"],
    dynamic_axes={"in": {2: "height", 3: "width"}, "out": {2: "height", 3: "width"}},
)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [24]:
import onnxruntime
import numpy as np

ort_session = onnxruntime.InferenceSession("./outputs/dynamic_input_cnn.onnx")
img = np.random.rand(1, 3, 20, 20).astype("float32")
ort_inputs = {"in": img}
ort_output = ort_session.run(output_names=["out"], input_feed=ort_inputs)[0]

img = np.random.rand(1, 3, 30, 30).astype("float32")
ort_inputs = {"in": img}
ort_output = ort_session.run(output_names=["out"], input_feed=ort_inputs)[0]

在上面的代码里，我们把input的后2个维度改为了: `20*20`以及`30*30`，可以正常推理。但如果我们改变batch这个维度，则会报错。

In [25]:
img = np.random.rand(2, 3, 20, 20).astype("float32")
ort_inputs = {"in": img}
ort_output = ort_session.run(output_names=["out"], input_feed=ort_inputs)[0]

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: in for the following indices
 index: 0 Got: 2 Expected: 1
 Please fix either the inputs or the model.

上面的错误告诉我们，在一个`in`的输入上维度无效，预期是1，输入是2

## 4.2 `torch.onnx.export`的使用技巧

通过学习之前的知识，我们基本掌握了 torch.onnx.export 函数的部分实现原理和参数设置方法，足以完成简单模型的转换了。但在实际应用中，使用该函数还会踩很多坑。这里把一些在实战中积累的一些经验分享给大家。

### 控制部分路径不导出到ONNX模型中

有些时候，我们希望模型在直接用 PyTorch 推理时有一套逻辑，而在导出的ONNX模型中有另一套逻辑。比如，我们可以把一些后处理的逻辑放在模型里，以简化除运行模型之外的其他代码。但使用`torch.onnx.is_in_onnx_export()`也有一些缺点：

使用 `is_in_onnx_export`确实能让我们方便地在代码中添加和模型部署相关的逻辑。但是，这些代码对只关心模型训练的开发者和用户来说很不友好，突兀的部署逻辑会降低代码整体的可读性。同时，`is_in_onnx_export`` 只能在每个需要添加部署逻辑的地方都“打补丁”，难以进行统一的管理。

In [2]:
import torch


class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = torch.nn.Conv2d(3, 3, 3)

    def forward(self, x):
        x = self.conv(x)
        # 我们仅在模型导出时把输出张量的数值限制在[0, 1]之间。
        if torch.onnx.is_in_onnx_export():
            x = torch.clip(x, 0, 1)
        return x

### 利用张量中断来进行模型静态化

在模型tracing时，一些对于Tensor的操作，比如`.item()`，对tensor进行遍历，通过List来创建Tensor，这些都会导致Tracing的中断，这些中间变量最终作为常量放在序列化模型中。

In [3]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x = x * x[0].item()
        return x, torch.Tensor([i for i in x])


model = Model()
dummy_input = torch.rand(10)
torch.onnx.export(model, dummy_input, "./outputs/a.onnx")

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



/tmp/ipykernel_4506/120990205.py:6: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x = x * x[0].item()
/tmp/ipykernel_4506/120990205.py:7: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  return x, torch.Tensor([i for i in x])
/tmp/ipykernel_4506/120990205.py:7: TracerWarning: torch.Tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return x, torch.Tensor([i for i in x

## 4.3 Pytorch对于ONNX算子的支持


在确保torch.onnx.export()的调用方法无误后，PyTorch 转 ONNX 时最容易出现的问题就是算子不兼容了。这里我们会介绍如何判断某个 PyTorch 算子在 ONNX 中是否兼容，以助大家在碰到报错时能更好地把错误归类。

在转换普通的torch.nn.Module模型时，PyTorch 一方面会用跟踪法执行前向推理，把遇到的算子整合成计算图；另一方面，PyTorch 还会把遇到的每个算子翻译成 ONNX 中定义的算子。在这个翻译过程中，可能会碰到以下情况：

* 该算子可以一对一地翻译成一个 ONNX 算子。
* 该算子在 ONNX 中没有直接对应的算子，会翻译成一至多个 ONNX 算子。
* 该算子没有定义翻译成 ONNX 的规则，报错。


ONNX的算子表格：https://github.com/onnx/onnx/blob/main/docs/Operators.md，我们可以在这个表格上查到所有ONNX算子从哪个opset_version来始支持的，以及后续的变更记录。

在Pytorch中，和ONNX有关的定义全部放在 torch.onxx 目录（https://github.com/pytorch/pytorch/tree/main/torch/onnx）中。其中，symbloic_opset{n}.py（符号表文件）即表示 PyTorch 在支持第 n 版 ONNX 算子集时新加入的内容。

如果我们想知道某个pytorch算子如何被翻译为ONNX算子的，就可以在上面的目录中进行搜索，看是哪个某个对应的opset版本中对应的定义的变化。比如：对于`_interpolate`算子，我们可以看到在`symbolic_helper.py`中的定义如下：

```python
return g.op(
            "Resize",
            input,
            empty_roi,
            empty_scales,
            size,
            coordinate_transformation_mode_s=coordinate_transformation_mode,
            cubic_coeff_a_f=-0.75,  # only valid when mode="cubic"
            mode_s=mode,  # nearest, linear, or cubic
            nearest_mode_s="floor",
        )
```

# 5. 解决部署中的算子适配问题

从以上pytorch模型转ONNX中可以看出，如果想要顺利转换，一般要满足以下的条件：

1. 我们要转换的计算过程，在Pytorch中都有对应的算子实现。
2. 存在方法把Pytorch算子映射成一个或多个ONNX算子
3. ONNX也存在对应的算子实现相应的功能。

可在实际部署中，这三部分的内容都可能有所缺失。其中最坏的情况是：我们定义了一个全新的算子，它不仅缺少 PyTorch 实现，还缺少 PyTorch 到 ONNX 的映射关系。我们可以考虑从以下三个方面，解决存在的问题：

1. 对于Pytorch算子支持的问题，我们可以考虑通过：1）组合现有的算子；2）添加torchscript算子；3）添加普通C++扩展算子。
2. 映射方面：1）为ATen算子添加符号函数；2）为Torchscript算子添加符号函数；3）封装成torch.autograd.Function并添加符号函数。
3. ONNX算子支持方面：1）使用现有的ONNX算子；2）定义新的ONNX算子。

那么，面对不同的情况时，就需要我们灵活地选用和组合这些方法。听起来是不是很复杂？别担心，本篇文章中，我们将围绕着三种算子映射方法，学习三个添加算子支持的实例，来理清如何为 PyTorch 算子转 ONNX 算子的三个环节添加支持。


## 5.1 为Aten算子定义转换规则

实际部署过程中，我们最常遇到的是，部分对Tensor的操作，在导出ONNX时，发现没有对应的映射。比如下面代码中，我们使用了`asinh`这个算子，它是一个ATen算子。而且这个算子在ONNX算子表中，从版本9开始就支持了。

PS. ATen 是 PyTorch 内置的 C++ 张量计算库，PyTorch 算子在底层绝大多数计算都是用 ATen 实现的。

In [2]:
import torch


class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return torch.asinh(x)


model = Model()
dummpy_input = torch.randn(2, 3)
torch.onnx.export(model, dummpy_input, "./outputs/asinh.onnx")

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



上面的代码在导出时报错了："Exporting the operator 'aten::asinh' to ONNX opset version 14 is not supported. "，这就是典型的没有在Pytorch中定义算子的映射规则。

那么我们可以就可以通过为Aten算子编写符号函数来指明映射规则。这里有几个要点：

1. 找到Pytorch中`asinh`这个ATen算子的接口定义
2. 查找对应的ONNX算子的名称与输入输出，建立对应。

`asinh`的接口定义为：`def asinh(input: Tensor, *, out: Optional[Tensor]=None) -> Tensor: ...`

我们定义的映射函数（符号函数）的接口要与`asinh`的接口完全一致。除了第一个参数是一个`torch._C.Graph`类型的参数`g`。在符号函数内部，通过`g.op`来返回一个ONNX中的算子定义。

In [3]:
from torch.onnx import register_custom_op_symbolic


def asinh_symbolic(g, input, *, out=None):
    # g.op的第一个参数是ONNX中算子的名称
    # 然后依次为这个ONNX算子的输入参数
    return g.op("Asinh", input)


# 向torch注意自定义的符号函数。在之前的版本中，这个函数为 register_op，新版本中已经找不到了
# 第一个参数是一个 "<domain>::<op>"
# 第三个参数指定了ONNX的算子版本
register_custom_op_symbolic("::asinh", asinh_symbolic, 9)

torch.onnx.export(
    model,
    dummpy_input,
    "./outputs/asinh.onnx",
    input_names=["in"],
    output_names=["out"],
    opset_version=9,
)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



值得注意的是，这里向第 9 号算子集注册，不代表较新的算子集（第 10 号、第 11 号……）都得到了注册。在示例中，我们先只向第 9 号算子集注册。

In [5]:
import onnxruntime
import numpy as np

ort_session = onnxruntime.InferenceSession("./outputs/asinh.onnx")
t = np.random.rand(2, 3).astype("float32")
ort_inputs = {"in": t}
ort_session.run(output_names=["out"], input_feed=ort_inputs)[0]

array([[0.01566388, 0.3456329 , 0.6761582 ],
       [0.14482886, 0.4115883 , 0.36688334]], dtype=float32)

## 5.2 为自定义Torch算子转ONNX


### 5.2.1 自定义的torchscript算子

对于一些比较复杂的运算，仅使用 PyTorch 原生算子是无法实现的。这个时候，就要考虑自定义一个 PyTorch 算子，再把它转换到 ONNX 中了。新增 PyTorch 算子的方法有很多，PyTorch 官方比较推荐的一种做法是添加 C++的扩展TorchScript 算子 。

由于用C++扩展一个支持torchScript算子比较复杂，现在我们可以用`torchvision`中的一个`DeformConv2d`来示例。`DeformConv2d`是torchvision中实现的一个算子扩展实现。

最终的算子是：`torch.ops.torchvision.deform_conv2d`

In [1]:
import torch
import torchvision


class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(3, 18, 3)
        self.conv2 = torchvision.ops.DeformConv2d(3, 3, 3)

    def forward(self, x):
        return self.conv2(x, self.conv1(x))


model = Model()
dummpy_input = torch.randn(1, 3, 32, 32)
# 可以trace成功
trace_model = torch.jit.trace(model, [dummpy_input])
# 但导出ONNX失败
torch.onnx.export(trace_model, dummpy_input, "./outputs/deform_conv2d.onnx")

/home/yangyansheng/miniconda3/envs/pyml/lib/python3.8/site-packages/torch/onnx/utils.py:825: UserWarning: no signature found for <torch.ScriptMethod object at 0x7f9fe7a6f540>, skipping _decide_input_format
  warnings.warn(f"{e}, skipping _decide_input_format")


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 1 ERROR ========================
ERROR: missing-custom-symbolic-function
ONNX export failed on an operator with unrecognized namespace torchvision::deform_conv2d. If you are trying to export a custom operator, make sure you registered it with the right domain and version.
None
<Set verbose=True to see more details>




UnsupportedOperatorError: ONNX export failed on an operator with unrecognized namespace torchvision::deform_conv2d. If you are trying to export a custom operator, make sure you registered it with the right domain and version.

也就是相当于现在C++扩展的算子已经写好了，需要把它映射到ONNX算子，按前面逻辑，我们只需要写符号函数，然后注册就可以了。这里遇到的问题是，我们发现ONNX目前也并不支持`DeformConv`，这就需要我们来自定义ONNX算子定义了。

我们在前面讲过，`g.op()` 是用来定义 ONNX 算子的函数。对于 ONNX 官方定义的算子，`g.op()` 的第一个参数就是该算子的名称。而对于一个自定义算子，`g.op()` 的第一个参数是一个带命名空间的算子名，比如：

```python
g.op("custom::deform_conv2d, ...)
```

In [9]:
from torch.onnx import register_custom_op_symbolic
from torch.onnx.symbolic_helper import parse_args


# parse_args标注每个参数的类型，这个是对于torchscript算子来说，符号函数要要求的
@parse_args("v", "v", "v", "v", "v", "i", "i", "i", "i", "i", "i", "i", "i", "none")
def symbolic(
    g,
    input,
    weight,
    offset,
    mask,
    bias,
    stride_h,
    stride_w,
    pad_h,
    pad_w,
    dil_h,
    dil_w,
    n_weight_grps,
    n_offset_grps,
    use_mask,
):
    # 这里为了简单，只用Input和offset构造了一个简单的ONNX算子
    return g.op("custom::deform_conv2d", input, offset)


register_custom_op_symbolic("torchvision::deform_conv2d", symbolic, 9)
torch.onnx.export(model, dummpy_input, "./outputs/deform_conv2d.onnx", opset_version=9)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 1 WARNING 0 ERROR ========================
1 WARNING were not printed due to the log level.



[W shape_type_inference.cpp:1920] Warning: The shape inference of custom::deform_conv2d type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)


其中，”::”前面的内容就是我们的命名空间。该概念和 C++ 的命名空间类似，是为了防止命名冲突而设定的。如果在 g.op() 里不加前面的命名空间，则算子会被默认成 ONNX 的官方算子。

PyTorch 在运行 g.op() 时会对官方的算子做检查，如果算子名有误，或者算子的输入类型不正确， g.op() 就会报错。为了让我们随心所欲地定义新 ONNX 算子，我们必须设定一个命名空间，给算子取个名，再定义自己的算子。

ONNX 是一套标准，本身并不包括实现。在这里，我们就简略地定义一个 ONNX 可变形卷积算子，而不去写它在某个推理引擎上的实现。


这段代码中，最令人疑惑的就是装饰器 `@parse_args` 了。简单来说，TorchScript 算子的符号函数要求标注出每一个输入参数的类型。比如”v”表示 Torch 库里的 value 类型，一般用于标注张量，而”i”表示 int 类型，”f”表示 float 类型，”none”表示该参数为空。具体的类型含义可以在 `torch.onnx.symbolic_helper.py`中查看。这里输入参数中的 input, weight, offset, mask, bias 都是张量，所以用”v”表示。后面的其他参数同理。我们不必纠结于 @parse_args的原理，根据实际情况对符号函数的参数标注类型即可。


### 5.2.2 自定义普通C++算子

对于非torchscript的C++算子，我们可以直接映射到python接口，然后放在`torch.autograd.Function`来调用。那么可以直接在自定义的`torch.autograd.Function`里定义`symbolic`符号函数来定义与ONNX算子的映射。

In [ ]:
class MyAddFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, a, b):
        return my_lib.my_add(a, b)

    @staticmethod
    def symbolic(g, a, b):
        two = g.op("Constant", value_t=torch.tensor([2]))
        a = g.op("Mul", a, two)
        return g.op("Add", a, b)

# 6. 修改和调试ONNX模型

我们有没有可能不用深度学习框架，自己用ONNX的API手动构造一个ONNX模型呢？或者当我们只拿到了一个ONNX模型时，我们如何对这个模型进行调试呢？

## 6.1 ONNX底层存储协议

ONNX底层使用Protobuf来序列化模型。我们可以在[ONNX源码](https://github.com/onnx/onnx/tree/main/onnx)下面看到一些.proto文件。神经网络本质上是一个计算图。计算图的节点是算子，边是参与运算的张量。而通过可视化 ONNX 模型，我们知道 ONNX 记录了所有算子节点的属性信息，并把参与运算的张量信息存储在算子节点的输入输出信息中。事实上，ONNX 模型的结构可以用类图大致表示如下：


<div align="center">
  <img src="./assets/onnx_proto.jpg" height="500"/> </div>

一个 ONNX 模型可以用 `ModelProto` 类表示。`ModelProto` 包含了版本、创建者等日志信息，还包含了存储计算图结构的 `graph`。`GraphProto` 类则由输入张量信息、输出张量信息、节点信息组成。张量信息 `ValueInfoProto` 类包括张量名、基本数据类型、形状。节点信息 `NodeProto` 类包含了算子名、算子输入张量名、算子输出张量名。 

下面我们使用ONNX API来构建一个`a * x + b`的模型。

In [4]:
import onnx
from onnx import helper
from onnx import TensorProto

# input and output Value
a = helper.make_tensor_value_info("a", TensorProto.FLOAT, [10, 10])
x = helper.make_tensor_value_info("x", TensorProto.FLOAT, [10, 10])
b = helper.make_tensor_value_info("b", TensorProto.FLOAT, [10, 10])
output = helper.make_tensor_value_info("output", TensorProto.FLOAT, [10, 10])

# Mul node
mul = helper.make_node("Mul", ["a", "x"], ["c"])
# Add node
add = helper.make_node("Add", ["c", "b"], ["output"])

# Build graph
# 这里 make_graph 的节点参数有一个要求：计算图的节点必须以拓扑序给出。
graph = helper.make_graph([mul, add], "linear_func", [a, x, b], [output])
model = helper.make_model(graph)

# save onnx model
onnx.checker.check_model(model)
print(model)
onnx.save(model, "./outputs/linear_func.onnx")

ir_version: 9
opset_import {
  version: 19
}
graph {
  node {
    input: "a"
    input: "x"
    output: "c"
    op_type: "Mul"
  }
  node {
    input: "c"
    input: "b"
    output: "output"
    op_type: "Add"
  }
  name: "linear_func"
  input {
    name: "a"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 10
          }
          dim {
            dim_value: 10
          }
        }
      }
    }
  }
  input {
    name: "x"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 10
          }
          dim {
            dim_value: 10
          }
        }
      }
    }
  }
  input {
    name: "b"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 10
          }
          dim {
            dim_value: 10
          }
        }
      }
    }
  }
  output {
    name: "output"
    type {
      tensor_type {
        elem_type: 

In [6]:
import onnxruntime
import numpy as np

ort_session = onnxruntime.InferenceSession("./outputs/linear_func.onnx")
a = np.random.randn(10, 10).astype(np.float32)
b = np.random.randn(10, 10).astype(np.float32)
x = np.random.randn(10, 10).astype(np.float32)
output = ort_session.run(["output"], {"a": a, "b": b, "x": x})[0]
assert np.allclose(output, a * x + b)

## 6.2 ONNX模型的读取与修改

In [10]:
import onnx

model = onnx.load("./outputs/linear_func.onnx")
# 获取模型中所有node
nodes = model.graph.node

In [12]:
nodes[0]

input: "a"
input: "x"
output: "c"
op_type: "Mul"

In [14]:
inputs = model.graph.input
inputs[0]

name: "a"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 10
      }
      dim {
        dim_value: 10
      }
    }
  }
}

In [17]:
# 将模型改为 a * x - b
model.graph.node[1].op_type = "Sub"

## 6.3 ONNX模型的调试 - 子图提取

在实际部署中，如果用深度学习框架导出的 ONNX 模型出了问题，一般要通过修改框架的代码来解决，而不会从 ONNX 入手，我们把 ONNX 模型当成一个不可修改的黑盒看待。 现在，我们已经深入学习了 ONNX 的原理，可以尝试对 ONNX 模型本身进行调试了。

<div align="center">
  <img src="./assets/whole_model.png" height="800"/> </div>

In [18]:
import torch


class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.convs1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 3, 3), torch.nn.Conv2d(3, 3, 3), torch.nn.Conv2d(3, 3, 3)
        )
        self.convs2 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 3, 3), torch.nn.Conv2d(3, 3, 3)
        )
        self.convs3 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 3, 3), torch.nn.Conv2d(3, 3, 3)
        )
        self.convs4 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 3, 3), torch.nn.Conv2d(3, 3, 3), torch.nn.Conv2d(3, 3, 3)
        )

    def forward(self, x):
        x = self.convs1(x)
        x1 = self.convs2(x)
        x2 = self.convs3(x)
        x = x1 + x2
        x = self.convs4(x)
        return x


model = Model()
input = torch.randn(1, 3, 20, 20)

torch.onnx.export(model, input, "./outputs/whole_model.onnx")

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [19]:
import onnx

onnx.utils.extract_model(
    "./outputs/whole_model.onnx",
    "./outputs/partial_model.onnx",
    input_names=["/convs1/convs1.2/Conv_output_0"],
    output_names=["/convs4/convs4.0/Conv_output_0"],
)

<div align="center">
  <img src="./assets/partial_model.png" height="500"/> </div>

子模型提取的实现原理：新建一个模型，把给定的输入和输出填入。之后把图的所有有向边反向，从输出边开始遍历节点，碰到输入边则停止，把这样遍历得到的节点做为子模型的节点。

所以如果我们指定的输出在计算时，所依赖到的输入，在整个计算路径上有一些未列入input_names的依赖，则会失败。



## 6.4 ONNX模型的调试 - 添加中间输出

我们依然利用`extract_model`接口，只需要在output_names里添加我们要输出的Tensor的名字即可。

In [21]:
import onnx

onnx.utils.extract_model(
    "./outputs/whole_model.onnx",
    "./outputs/multi_output_model.onnx",
    input_names=["/convs1/convs1.2/Conv_output_0"],
    output_names=[
        "/convs4/convs4.0/Conv_output_0",
        "/convs3/convs3.1/Conv_output_0",
        "/convs2/convs2.1/Conv_output_0",
    ],
)

<div align="center">
  <img src="./assets/multioutput_model.png" height="500"/> </div>